In [ ]:
cmap = ['blue', 'orange', 'green', 'red', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan']

In [ ]:
import pprint as pp
import numpy as np
import time
import os
import sys
import random


import matplotlib.pyplot as plt
%matplotlib inline 
import matplotlib
from matplotlib.patches import Patch

import shutil
from numpy import genfromtxt
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable

torch.set_default_dtype(torch.float64)

import itertools
import numpy.linalg  as lin

import cProfile, pstats

from collections import OrderedDict
torch.set_num_threads(1) #cpu num
from fractions import Fraction
import seaborn as sns
import pandas as pd
import math
from collections import defaultdict

In [ ]:
def avg_alpha(_aa):
    aa_ep = defaultdict(list)
    for i, x in enumerate(_aa):
        aa_ep[math.ceil(x[0])].append(i)
        
    _aa = np.array(_aa)
        
    _a = []
    _areal = []
    _amax = []
    
    for key in sorted(aa_ep.keys()):
        ind = aa_ep[key]
        _a.append(np.mean(_aa[ind,2]))
        _areal.append(np.mean(_aa[ind,3]))
        _amax.append(np.mean(_aa[ind,4]))
    return list(sorted(aa_ep.keys())),_a, _areal, _amax
    

def fillup(_epp,_ep,_l):
    
    for ind in range(1,len(_epp)):
        if _epp[ind] not in _ep:
            _l.insert(ind,_l[ind-1])
    return _l    

def runmax(_l):
    
    return [max(_l[:i+1]) for i in range(len(_l))]


In [ ]:
DNAME = ['ijcnn1','rcv1','real-sim','news20','covtype']

In [ ]:
%cd ../SenseLogs

In [ ]:
! pwd

In [ ]:
df={}
adf={}
for dname in DNAME:    

    logfolder = '%s/reg/ai_sarah/'%dname
    epp={}
    for f in os.listdir(logfolder):
        if '.tar' not in f:
            continue
        else:
            temp = torch.load(logfolder+f)
            parm = temp['parm']
            hist = np.array(temp['hist'])
            stat = np.array(temp['stat'])
            o = parm[2]
            if o not in epp:
                epp[o]=[]

            ep = list(np.round(hist[:,0]))
            epp[o]+=ep

    EPP={}
    for o in sorted(epp.keys()):
        EPP[o] = sorted(list(set(epp[o])))

    DF = []
    aDF=[]

    for f in os.listdir(logfolder):
        if '.tar' not in f:
            continue
        else:
            temp = torch.load(logfolder+f)
            parm = temp['parm']
            BS = parm[0]
            seed = parm[1]
            gamma = parm[2]
            hist = np.array(temp['hist'])
            stat = np.array(temp['stat'])
            alpha = temp['alpha']
            O = gamma

            ep = list(np.round(hist[:,0]))
            loss = fillup(EPP[O],ep,list(hist[:,1]))
            grad = fillup(EPP[O],ep,list(hist[:,2]))
            test = fillup(EPP[O],ep,list(hist[:,3]))
            test = runmax(test)
            innerT = fillup(EPP[O],ep,list(stat[:,2]))
            outerT = fillup(EPP[O],ep,list(stat[:,1]))
            timeT = fillup(EPP[O],ep,list(stat[:,3]))
            
            aEP,aL,arL,amL = avg_alpha(alpha)
            
            DF+=[[upper_bound+heuristic, gamma, BS, seed, e,oT,iT,tT,l,g,t] for e,oT,iT,tT,l,g,t\
                in zip(EPP[O],outerT,innerT,timeT,loss,grad,test)]
            
            aDF+=[[upper_bound+heuristic, gamma, BS, seed, e, 'alpha', ai] for e,ai\
                in zip(aEP,aL)]
            aDF+=[[upper_bound+heuristic, gamma, BS, seed, e, 'alpha_real', ali] for e,ali\
                in zip(aEP,arL)]
            aDF+=[[upper_bound+heuristic, gamma, BS, seed, e, 'alpha_max', ami] for e,ami\
                in zip(aEP,amL)]

    df[dname] = pd.DataFrame(data=DF,columns=['algo','gamma','BS','seed',\
                                       'ep','outer','inner','time','loss','grad','test'])
    df[dname] = df[dname].sort_values(by=['algo'])
    
    adf[dname] = pd.DataFrame(data=aDF,columns=['algo','gamma','BS','seed',\
                                       'ep','type','alpha'])
    adf[dname] = adf[dname].sort_values(by=['algo','type'])

In [ ]:
sns.set()
sns.set(context="paper",font='serif')
sns.set_style("ticks")
fig, ax = plt.subplots(1,5,figsize=(15,3),sharex=True,sharey=False)

palette = sns.color_palette()[:6]
palette = cmap[1:2]+cmap[0:1]+cmap[2:6]
markers = ('v', 'o', '^', '<', '>', '8')
dashes = [(2,2)]*6

for d in range(5):
    if d==0:
        LEGEND='full'
    else:
        LEGEND=False
    subdf = df[DNAME[d]]
    lp=sns.lineplot(x='ep',y='grad',dashes=dashes,palette=palette,\
                 markers=markers,zorder=-1,markevery=5,markersize=8,lw=2.0,\
                    legend=LEGEND,hue='gamma',ci=95,style='gamma',\
                    data=subdf,ax=ax[d])
    ax[d].set_yscale('log') 
    ax[d].grid('both')
    if d==0:
        ax[d].set_ylabel(r'$||\nabla P(w)||^2$',fontsize=12)
    else:
        ax[d].set_ylabel('')
    ax[d].set_xlabel('Effective Pass',fontsize=12)
    
handles, labels = ax[0].get_legend_handles_labels()
labels = [r'$\gamma$']+[str(Fraction(n)) for n in labels[1:]]
ax[0].legend(handles=handles,labels=labels,fontsize=10,loc='lower left',fancybox=True)
fig.tight_layout()
plt.show()

In [ ]:
exit(0)